# 1D CODE

In [1]:
import numpy as np
import matplotlib
import math
import time
import pickle


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import itertools


import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML
import tensorflow_probability as tfp


%matplotlib inline

## Run auxiliary functions:

In [2]:
%run ./auxiliary_func_noprint.ipynb

## Choose input/output dimensions and look at the examples

In [3]:
## Allowed input/output values in this preliminary version are (ni=2 no=1) (ni=3 no=1) (ni=3 no=2) and (ni=1 no=1) 

ni=1 #number of input variables  
no=1 #number of output solutions 

#LOOK AT THE PDE DICTIONARIES MATCHING THE DIMENSION REQUIREMENTS
#################################################################################
### outputfunction is named u, v
### first derivatives are named dudt, dudx, dudy
### pure second derivatives are named dduddt, dduddx, dduddy
### mixed second derivative (as ddudtdx) need to be coded in the loss function, we will provide an automatic derivation in future updates.
############## NOTATION #####################
## eomdict: impose PDE constraints. The form needs to be '[ eq1 , eq2 ]' 
## ICdict: Boundary conditions on t belonging to [t0,tL].
##         The form needs to be [ '[ u - u(t0,x,y),dudt - dudt(t0,x,y)]' , '[u - u(tL,x,y),dudt - dudt(tL,x,y)]' ] 
##         If no condition needs to be provided on some boundary just replace (u - u(t,x,y0) ) with  [none].
## boardx: Boundary conditions on x belonging to [x0,xL].
##         The form needs to be [ '[u - u(t,x0,y),dudx - dudx(t,x0,y)]' , '[u - u(t,xL,y),dudx - dudx(t,xL,y)]' ]   
##         If no condition needs to be provided on some boundary just replace (u - u(t,x,y0) ) with  [none].
## boardy: Boundary conditions in y belonging to [y0,yL].
##         The form needs to be [ '[u - u(t,x,y0),dudy - dudy(t,x,y0)]' , '[u - u(t,x,yL),dudy - dudy(t,x,yL)]' ] 
##         If no condition needs to be provided on some boundary just write  [none].
## wdict:  weights to be used in loss function
##         The form needs to be [w_bulk,w_IC,w_board]

eqname_dict=eqname_gen(ni,no)
analytic=eqname_analytic(ni,no)
sol=sol_analytic(ni,no)

if ni==1:
    eomdict,ICdict=dictgen(ni,no)
elif ni==2:
    eomdict,ICdict,boardx,wdict=dictgen(ni,no)
elif ni==3:
    eomdict,ICdict,boardx,boardy,wdict=dictgen(ni,no)

eomdict


{'oscillon': '[dduddt-0.5**2*u+2.*u**3.]',
 'mat': '[dduddt+(1.-0.4*tf.cos(2.*t))*u]',
 'exp': '[dudt+0.5*u]',
 'wave': '[dduddt+25.*u]',
 'dho': '[dduddt+0.5*dudt+25.*u]',
 'linear': '[dudt-1.]',
 'delay': '[dudt-0.5*u+1.*du]',
 'gaussian': '[dudt+0.2*t*u]',
 'stiff': '[dudt+21.*u-tf.exp(-t)]',
 'twofreq': '[dduddt+u+2.*tf.cos(5.*t)+6.*tf.sin(10.*t)]'}

In [4]:
#####################################################
#### Choose the equation of motion to be studied
#####################################################
### 3to1 D equations studied: ['waveb','wave','wavet', 'poisson1', 'poisson2', 'poisson3','heat1','heat2']
### 3to2 D equations studied: ['GT','LO','NS']
### 2to1 D equations studied: [{'wave','waveN','twave','heat','heat2f','heat0','poisson','poisson1','AdvectionDiffusion','Burgers','poisson_par','par']
### 1to1 D equations studied: [{'oscillon', 'mat', 'exp', 'wave', 'dho', 'linear', 'delay', 'gaussian', 'stiff', 'twofreq'}]
#####
##Create the list of coordinate boundaries, i.e. boundaries=[t_0,t_max,x_0,x_max,y_0,y_max]
#### Boudaries used in the paper
#bd=[0.,1.,0.,1.,0.,1.]               # 3to1 D
#bd=[0.,1.,0.,2.*np.pi,0.,2.*np.pi]   #'GT'
#bd=[0.1,1.,-2.,2.,-2.,2.]            #'LO'
#bd=[0.,10.,0.,1.,0.,1.]              #'NS'
#bd=[0.,1.,0.,1.]                     # 2to1 D
#bd=[0.,20.]                          # 1to1 D

bd=[0.,20.]



## define number of points to be sampled in the bulk, the IC and on each boundary surface
## for the 1D case (n_bulk, n_IC) will evaluate to (batch_sz-2,2) for the training part of adam
n_bulk=2000
n_IC=1
batch_sz=256


##Initialise the network
n_l=35 #number of neurons per branch

#create base frequencies to initialize the network
pi=tf.constant(np.pi,dtype=tf.float32)
none=tf.constant(0.,dtype=tf.float32)
freq_t=base_freq(ni,bd)
step=1.

model=dNNsolve(ni,no,n_l,bd,step)

model.save_weights(f"model_seq_{ni}D_{n_l}nodes_randomMiniBatch.h5")


## Training and plots

In [5]:
epochs_adam=1001  

f= open("1to1_results_randomMiniBatch.txt","w+")
oneD_dict=['oscillon','mat','exp','wave','dho','linear','delay','gaussian','stiff','twofreq']


for eom in oneD_dict:
    
    #Mini-batch sized prototype dataset for adam
    X_data = (random_sampling(ni))(bd,batch_sz-2,1)
    #Define ic, bc and bulk counters (I_t0,I_tL,I_bulk)
    I=counters(ni)(bd,X_data)
    [I_t0,I_tL,I_bulk]= tf.split(I,3,axis=1)
    fake_output=tf.concat([X_data,I],axis=1)
    step=2.
    inputs=[X_data]
    
    #Larger dataset for BFGS
    X_data_bfgs = (random_sampling(ni))(bd,n_bulk,n_IC)
    I=counters(ni)(bd,X_data_bfgs)
    fake_output_bfgs=tf.concat([X_data_bfgs,I],axis=1)

    ########################
    start=time.time() 
    #initialization of models
    model.load_weights(f"model_seq_{ni}D_{n_l}nodes_randomMiniBatch.h5") #Restore weights to original values at each iteration
    loss_fun=to_loss_1to1_minibatch(eom) 
    loss_fun_bfgs=to_loss_1to1(eom)  


    # an educated guess for the oscillon central amplitude
    if ni==1 and eom=='oscillon':
        u0guess=tf.constant(0.6)

    #Initialise BFGS method and compile the model
    func = function_factory(model, loss_fun_bfgs, fake_output_bfgs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.1),
                           loss=loss_fun,run_eagerly=False)

    #Training with Adam
    hist = model.fit(x=inputs, y=fake_output, batch_size = batch_sz, epochs=epochs_adam, verbose=0, callbacks = 
                       [Print_Loss_Every_so_many_Epochs_1D(),
                       tf.keras.callbacks.ReduceLROnPlateau(monitor='loss', mode='min', factor=1./2., patience=30, min_lr=1e-4)])


    #Training with BFGS
    init_params = tf.dynamic_stitch(func.idx, model.trainable_variables)
    results = tfp.optimizer.bfgs_minimize(value_and_gradients_function=func, initial_position=init_params,tolerance=1e-20, max_iterations=5000)    
    func.assign_new_model_parameters(results.position)



    #Store loss data
    loss_adam=np.array(hist.history['loss'],dtype=np.float32) 
    history_lbfgs=np.array(func.history)
    hist_lbfgs=np.array(history_lbfgs[:,0])
    loss=np.concatenate((loss_adam,hist_lbfgs))
    end=time.time()

    n_plt=300
    pred, smse, tplt=points_plt_mse(ni,no,bd,model,sol) 


    eps=1e-20 ##small parameter to avoid inf values
    f.write('\n EOM: ' + eom)
    f.write('\n Neurons per branch:  %s' % (n_l))
    f.write('\n Epochs:  %s' % (loss.size))
    f.write('\n Total Time: %.4f' % (end-start))
    f.write('\n Final log10(Loss Adam): %.4f' % (np.log10(loss_adam[-1])+eps))
    f.write('\n Final log10(Loss): %.4f' % (np.log10(loss[-1])+eps))
    f.write('\n Log10 of  Sqrt Mean squared error:  %.4f' % (np.log10(smse+1e-20)+eps))
    f.write('\n log10(Loss bulk): %.4f log10(Loss IC): %.4f log10(Loss board): %.4f' % (np.log10(eps+np.array(history_lbfgs[:,1])[-1]),np.log10(eps+np.array(history_lbfgs[:,2])[-1]),np.log10(eps+np.array(history_lbfgs[:,3])[-1])))
    f.write('\n #######################################################')

f.close()

Epoch 0:	   log10 loss:   -0.37 
Epoch 100:	   log10 loss:   -0.65 
Epoch 200:	   log10 loss:   -1.20 
Epoch 300:	   log10 loss:   -1.35 
Epoch 400:	   log10 loss:   -1.68 
Epoch 500:	   log10 loss:   -1.77 
Epoch 600:	   log10 loss:   -1.78 
Epoch 700:	   log10 loss:   -1.79 
Epoch 800:	   log10 loss:   -1.78 
Epoch 900:	   log10 loss:   -1.77 
Epoch 1000:	   log10 loss:   -1.79 
Epoch 0:	   log10 loss:   -0.45 
Epoch 100:	   log10 loss:   -0.82 
Epoch 200:	   log10 loss:   -1.09 
Epoch 300:	   log10 loss:   -1.36 
Epoch 400:	   log10 loss:   -1.41 
Epoch 500:	   log10 loss:   -1.43 
Epoch 600:	   log10 loss:   -1.46 
Epoch 700:	   log10 loss:   -1.42 
Epoch 800:	   log10 loss:   -1.43 
Epoch 900:	   log10 loss:   -1.44 
Epoch 1000:	   log10 loss:   -1.42 
Epoch 0:	   log10 loss:   -0.45 
Epoch 100:	   log10 loss:   -1.07 
Epoch 200:	   log10 loss:   -1.44 
Epoch 300:	   log10 loss:   -1.70 
Epoch 400:	   log10 loss:   -2.29 
Epoch 500:	   log10 loss:   -2.73 
Epoch 600:	   log10 loss

/Users/Garfield/anaconda3/envs/tensor_flow/lib/python3.7/site-packages/ddeint/ddeint.py:145: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return np.array([g(tt[0])] + results)


Epoch 0:	   log10 loss:   -0.45 
Epoch 100:	   log10 loss:   -0.91 
Epoch 200:	   log10 loss:   -1.29 
Epoch 300:	   log10 loss:   -2.13 
Epoch 400:	   log10 loss:   -2.58 
Epoch 500:	   log10 loss:   -3.02 
Epoch 600:	   log10 loss:   -3.29 
Epoch 700:	   log10 loss:   -3.62 
Epoch 800:	   log10 loss:   -3.64 
Epoch 900:	   log10 loss:   -3.79 
Epoch 1000:	   log10 loss:   -3.90 
Epoch 0:	   log10 loss:   -0.37 
Epoch 100:	   log10 loss:   -0.22 
Epoch 200:	   log10 loss:   -0.41 
Epoch 300:	   log10 loss:   -0.59 
Epoch 400:	   log10 loss:   -0.49 
Epoch 500:	   log10 loss:   -0.87 
Epoch 600:	   log10 loss:   -1.02 
Epoch 700:	   log10 loss:   -0.96 
Epoch 800:	   log10 loss:   -0.91 
Epoch 900:	   log10 loss:   -0.55 
Epoch 1000:	   log10 loss:   -0.64 
Epoch 0:	   log10 loss:    0.43 
Epoch 100:	   log10 loss:    0.35 
Epoch 200:	   log10 loss:    0.34 
Epoch 300:	   log10 loss:    0.33 
Epoch 400:	   log10 loss:    0.33 
Epoch 500:	   log10 loss:    0.32 
Epoch 600:	   log10 loss